## MLflow's Model Registry

In [1]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [4]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
client.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomzamp/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1747211534647, experiment_id='1', last_update_time=1747211534647, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomzamp/mlops-zoomcamp/02-experiment-tracking/mlruns/0', creation_time=1747211534639, experiment_id='0', last_update_time=1747211534639, lifecycle_stage='active', name='Default', tags={}>]

In [5]:
client.create_experiment(name="my-cool-experiment")

'2'

Let's check the latest versions for the experiment with id `1`...

In [6]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [7]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: b4412345fe2a482cab6e998b81090b51, rmse: 6.3184
run id: 54f7d4026c9f4412b8d0c05c29cae01a, rmse: 6.3184
run id: 00b33b8d2fe44ccab284e5b73a7d5cb1, rmse: 6.3184
run id: 7992cfb161494d08a40afe0c4f93cded, rmse: 6.3292
run id: 6400d610052e4f76a30fa40abcbbe5ba, rmse: 6.3896


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [9]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [10]:
run_id = "b4412345fe2a482cab6e998b81090b51"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Successfully registered model 'nyc-taxi-regressor'.
Created version '1' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1747299531503, current_stage='None', description=None, last_updated_timestamp=1747299531503, name='nyc-taxi-regressor', run_id='b4412345fe2a482cab6e998b81090b51', run_link=None, source='/workspaces/mlops-zoomzamp/mlops-zoomcamp/02-experiment-tracking/mlruns/1/b4412345fe2a482cab6e998b81090b51/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [11]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Staging
version: 2, stage: None


/tmp/ipykernel_6338/669935608.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [14]:
from mlflow.tracking import MlflowClient

client = MlflowClient()
model_name = "nyc-taxi-regressor"

# Retrieve the latest versions of the model
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

    version_info = client.get_model_version(name=model_name, version=version.version)
    print(f"  - Registered at: {version_info.creation_timestamp}")
    print(f"  - Tags: {version_info.tags}")


version: 1, stage: Staging
  - Registered at: 1747299531503
  - Tags: {'env': 'staging'}
version: 2, stage: Production
  - Registered at: 1747299709944
  - Tags: {'env': 'production'}


/tmp/ipykernel_6338/1719468604.py:7: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [20]:
# Add aliases to the model version
model_version = 2
new_stage = "testing"
client.set_registered_model_alias(
    name=model_name,
    version=model_version,
    alias=new_stage
)

In [ ]:
# Add Description to the model version
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=['testing'], creation_timestamp=1747299709944, current_stage='Production', description='The model version 2 was transitioned to testing on 2025-05-15', last_updated_timestamp=1747300618522, name='nyc-taxi-regressor', run_id='b4412345fe2a482cab6e998b81090b51', run_link='', source='models:/nyc-taxi-regressor/1', status='READY', status_message=None, tags={'env': 'production'}, user_id=None, version=2>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [23]:
from sklearn.metrics import root_mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_csv(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": root_mean_squared_error(y_test, y_pred)}

In [25]:
df = read_dataframe("data/green_tripdata_2021-03.csv.gz")

/tmp/ipykernel_6338/633860941.py:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


In [26]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/workspaces/mlops-zoomzamp/mlops-zoomcamp/02-experiment-tracking/preprocessor'

In [27]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [28]:
X_test = preprocess(df, dv)

In [29]:
target = "duration"
y_test = df[target].values

In [30]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

CPU times: user 16.3 s, sys: 267 ms, total: 16.5 s
Wall time: 5.31 s


{'rmse': 6.2702965482607915}

In [31]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

CPU times: user 16 s, sys: 27 ms, total: 16 s
Wall time: 4.37 s


{'rmse': 6.2702965482607915}

In [32]:
client.transition_model_version_stage(
    name=model_name,
    version=2,
    stage="Production",
    archive_existing_versions=True
)

/tmp/ipykernel_6338/1703833061.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=['testing'], creation_timestamp=1747299709944, current_stage='Production', description='The model version 2 was transitioned to testing on 2025-05-15', last_updated_timestamp=1747301065549, name='nyc-taxi-regressor', run_id='b4412345fe2a482cab6e998b81090b51', run_link='', source='models:/nyc-taxi-regressor/1', status='READY', status_message=None, tags={'env': 'production'}, user_id=None, version=2>